## Importamos lo necesario para poder interactuar de a primeras con nuestro modelo y la herramienta de busqueda

In [3]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

llm=ChatOpenAI(model='gpt-4o-mini',temperature=0.3)
search_tool = TavilySearchResults(max_results=5)

print(llm.invoke("Cual es la capital de venezuela"))
print(search_tool.invoke("Cual es la capital de españa?"))

content='La capital de Venezuela es Caracas.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BKoYr8XLbUKzZhZf50fzPND6eqM76', 'finish_reason': 'stop', 'logprobs': None} id='run-8a0e7bdd-1f50-4b76-80d9-8650348553a7-0' usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
[{'title': 'Principales Ciudades de España', 'url': 'https://www.camaraofespanola.org/principales-ciudades-de-espana/', 'content': 'Madrid es la capital de España. Con una población de cuatro millones 726 mil h

## Agente de busqueda preconstruido con REACT

In [ ]:
## Un agente react es una clase preconstruida de langgraph el cual tiene una cadena de pensamiento en base a los recursos que tiene, de modo tal que:
##s un tipo de agente basado en la técnica ReAct (Reasoning + Acting) que combina razonamiento paso a paso con ejecución de acciones. Viene como uno de los agentes preconstruidos (prebuilt agents) que puedes usar directamente con LangGraph y LangChain.
## Un ReAct Agent:

##Razona paso a paso ("Thought").

##Toma acciones concretas usando herramientas disponibles ("Action").

##Usa el resultado de la acción ("Observation") para razonar más y continuar.

##Repite este ciclo hasta alcanzar una respuesta final.

from typing import Literal
from typing_extensions import TypedDict
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage
from langchain.memory import  PostgresChatMessageHistory

class State(MessagesState):
    next: str
    
    
from langgraph.prebuilt import create_react_agent

research_agent = create_react_agent(
    model=llm,
    tools=[search_tool],
    prompt="Eres un agente experto en la búsqueda de información para responder preguntas usando siempre fuentes fiables, razonando de manera inteligente sobre tu respuesta."
)

print(research_agent.invoke({
    "messages": [HumanMessage(content="¿Cuál es la capital de España?")]
}))



{'messages': [HumanMessage(content='¿Cuál es la capital de España?', additional_kwargs={}, response_metadata={}, id='ae381324-83e1-4084-aee6-67360744f8a6'), AIMessage(content='La capital de España es Madrid. Es la ciudad más grande del país y un importante centro cultural, político y económico.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 114, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_64e0ac9789', 'id': 'chatcmpl-BKopoa6Ei79bFiGk42Aj4pPAlGKTq', 'finish_reason': 'stop', 'logprobs': None}, id='run-8c43d70e-875c-4ee6-b3b2-eabc1e93b6a8-0', usage_metadata={'input_tokens': 114, 'output_tokens': 26, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

#### Perfecto!!, el agente funciona correctamente, ahora hagamos nuestra primera función util para el grafo final

In [11]:
## Definimos a nuestro agente supervisor
members=["researcher", "sql_agent"]
options=members+['FINISH']

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Trabajador a enrutar al siguiente. Si no se necesitan trabajadores, enrutar a FINALIZAR."""

    next: Literal[*options]
    
def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})    

##Usamos el estado para obtener la consulta y lo dirigimos al supervisor una vez definido
def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )

## Va agarrando forma!! ahora hagamos lo verdaderamente complejo, conectemos el supervisor a nuestro agente de sql